<a href="https://colab.research.google.com/github/amanjain487/learning_CV/blob/main/Object%20Detection/Transformers/WIDER_Face_Face_Detection_using_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

os.chdir('/content/drive/MyDrive/')
os.makedirs('Wider_Face/dataset', exist_ok=True)
os.chdir('Wider_Face/dataset')
print(os.getcwd())

/content/drive/MyDrive/Wider_Face/dataset


In [ ]:
import zipfile

os.chdir("/content/drive/MyDrive/Wider_Face/dataset")

if not os.path.exists('WIDER_train'):
  with zipfile.ZipFile('WIDER_train.zip') as train_zip:
      train_zip.extractall()
      !cp -r /content/drive/MyDrive/Wider_Face/dataset/WIDER_train/images/* /content/drive/MyDrive/Wider_Face/dataset/WIDER_train/

if not os.path.exists('WIDER_val'):
  with zipfile.ZipFile('WIDER_val.zip') as test_zip:
      test_zip.extractall()
      !cp -r /content/drive/MyDrive/Wider_Face/dataset/WIDER_val/images/* /content/drive/MyDrive/Wider_Face/dataset/WIDER_val/

if not os.path.exists('wider_face_split'):
  with zipfile.ZipFile('wider_face_split.zip') as wider_face_split_zip:
      wider_face_split_zip.extractall()


In [ ]:
import re
import cv2
import json
import itertools
import numpy as np
from glob import glob
import scipy.io as sio
from pycocotools import mask as cocomask
from PIL import Image

os.chdir('/content/drive/MyDrive/Wider_Face/dataset')

MAX_N = 100

categories = [
    {
        "supercategory": "none",
        "name": "face",
        "id": 0
    }
]

phases = ["train", "val"]
for phase in phases:
    root_path = "WIDER_{}/images/".format(phase)
    gt_path = os.path.join("wider_face_split/wider_face_{}.mat".format(phase))
    json_file = "{}.json".format(phase)

    gt = sio.loadmat(gt_path)
    event_list = gt.get("event_list")
    file_list = gt.get("file_list")
    face_bbox_list = gt.get("face_bbx_list")

    res_file = {
        "categories": categories,
        "images": [],
        "annotations": []
    }

    annot_count = 0
    image_id = 0
    processed = 0
    for event_idx, path in enumerate(event_list):
        base_path = path[0][0]
        for file_idx, img_name in enumerate(file_list[event_idx][0]):
            file_path = img_name[0][0]
            face_bbox = face_bbox_list[event_idx][0][file_idx][0]
            num_boxes = face_bbox.shape[0]

            if num_boxes > MAX_N:
                continue

            img_path = os.path.join(root_path, base_path, file_path + ".jpg")
            filename = os.path.join(base_path, file_path + ".jpg")

            img = Image.open(img_path)
            img_w, img_h = img.size
            img_elem = {"file_name": filename,
                        "height": img_h,
                        "width": img_w,
                        "id": image_id}

            res_file["images"].append(img_elem)

            for i in range(num_boxes):
                xmin = int(face_bbox[i][0])
                ymin = int(face_bbox[i][1])
                xmax = int(face_bbox[i][2]) + xmin
                ymax = int(face_bbox[i][3]) + ymin
                w = xmax - xmin
                h = ymax - ymin
                area = w * h
                poly = [[xmin, ymin],
                        [xmax, ymin],
                        [xmax, ymax],
                        [xmin, ymax]]

                annot_elem = {
                    "id": annot_count,
                    "bbox": [
                        float(xmin),
                        float(ymin),
                        float(w),
                        float(h)
                    ],
                    "segmentation": list([poly]),
                    "image_id": image_id,
                    "ignore": 0,
                    "category_id": 0,
                    "iscrowd": 0,
                    "area": float(area)
                }

                res_file["annotations"].append(annot_elem)
                annot_count += 1

            image_id += 1

            processed += 1

    with open(json_file, "w") as f:
        json_str = json.dumps(res_file)
        f.write(json_str)

    print("Processed {} {} images...".format(processed, phase))
print("Done.")

KeyboardInterrupt: ignored

In [ ]:
import os

os.chdir('/content/drive/MyDrive/Wider_Face/')
print(os.getcwd())

/content/drive/MyDrive/Wider_Face


In [ ]:
! git clone https://github.com/facebookresearch/detr.git

fatal: destination path 'detr' already exists and is not an empty directory.


In [ ]:
os.chdir('/content/drive/MyDrive/Wider_Face/detr')

os.makedirs('weights', exist_ok=True)


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Wider_Face/detr')

In [ ]:
! python main.py --coco_path ../dataset/ --output_dir output --epoch 5 --resume weights/detr-r50-e632da11.pth --batch_size 4

# ! python main.py --coco_path ../dataset/ --output_dir output --epoch 11 --resume output/checkpoint.pth --batch_size 4


Not using distributed mode
git:
  sha: eb9f7e03ed8e2ed2cd55528989fe7df890bc3fc0, status: has uncommited changes, branch: master

Namespace(aux_loss=True, backbone='resnet50', batch_size=4, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path=None, coco_path='../dataset/', dataset_file='coco', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', distributed=False, dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=5, eval=False, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=0.0001, lr_backbone=1e-05, lr_drop=200, mask_loss_coef=1, masks=False, nheads=8, num_queries=100, num_workers=2, output_dir='output', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='weights/detr-r50-e632da11.pth', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, weight_decay=0.0001, world_size=1)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoi

In [ ]:
%matplotlib inline
!python test.py --data_path ../dataset/WIDER_val/ --resume output/checkpoint.pth

processing...0_Parade_marchingband_1_1004.jpg
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
done
